In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import re
from tqdm.notebook import tqdm
from numba import jit

In [2]:
df = pd.read_stata("../ethiopia_data/data/processed/full_panel.dta")

/usr/lib/python3.9/site-packages/pandas/io/stata.py:1457: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [3]:
trajectories = (
    df
    .dropna(subset= ['impmaize'])
    .groupby(['holder_id'])['impmaize']
    .agg(trajectories = list)
    .assign(len_traj = lambda df: df['trajectories'].apply(lambda x: len(x)))
    .query("len_traj == 3")
    .drop(['len_traj'], axis=1)
    .assign(trajectories = lambda df: df['trajectories'].astype(str))
    .pipe(pd.get_dummies)
    .rename(lambda x: x.replace('.0', '').replace(',', '').replace('[', '').replace(']', '').replace(' ', ''), axis=1)
    )

# merge with df

merged_df = (
    df
    .merge(trajectories, 
           left_on= ['holder_id'], 
           right_index=True)

    )



In [6]:
reg_dict = {}

outcomes = merged_df.columns[merged_df.columns.str.contains('YIELD')].tolist()
h_switchers = merged_df.columns[merged_df.columns.str.contains("trajectories_")][1:7].tolist()
h_switchers_int = [f"{i}:impmaize" for i in h_switchers]
h_no_always = merged_df.columns[merged_df.columns.str.contains("trajectories_")][0:7].tolist()


for y in outcomes:
    
    merged_df_dropna = merged_df.dropna(subset= [y] + ['impmaize'] + h_no_always + h_switchers)

    reg_dict[y] = smf.ols(f"np.arcsinh({y}) ~ -1 + {' + '.join(h_no_always)} + {' + '.join(h_switchers_int)}", 
                          data = merged_df_dropna)

In [7]:
# Now run weak-id test
reg_res_dict = {}

for y, mod in reg_dict.items():
    merged_df_dropna = merged_df.dropna(subset= [y] + ['impmaize'] + h_no_always + h_switchers)

    print(f"Trying {y}")
    res = mod.fit(cov_type = 'cluster', cov_kwds = {'groups' : merged_df_dropna['holder_id']})
    reg_res_dict[y] = res


Trying YIELD_cropcutfresh
Trying YIELD_cropcutdry
Trying YIELD_cropcutfresh_tr
Trying YIELD_cropcutdry_tr
Trying YIELD_selfr
Trying YIELD_selfr_tr


In [8]:
reg_res_dict['YIELD_cropcutdry_tr'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                       OLS Regression Results                                       
====================================================================================================
Dep. Variable:     np.arcsinh(YIELD_cropcutdry_tr)   R-squared (uncentered):                   0.243
Model:                                         OLS   Adj. R-squared (uncentered):              0.240
Method:                              Least Squares   F-statistic:                              81.11
Date:                             Fri, 26 Nov 2021   Prob (F-statistic):                   6.11e-151
Time:                                     10:26:13   Log-Likelihood:                         -8711.6
No. Observations:                             3311   AIC:                                  1.745e+04
Df Residuals:                                 3298   BIC:                                  1.753e+04
Df Model:                                       13                                                  
Covariance Type:                           cluster                                                  
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
trajectories_000              2.0528      0.074     27.792      0.000       1.908       2.198
trajectories_001              2.0115      0.312      6.457      0.000       1.401       2.622
trajectories_010              2.1948      0.362      6.064      0.000       1.485       2.904
trajectories_011              1.2052      0.303      3.972      0.000       0.611       1.800
trajectories_100              2.0762      0.372      5.584      0.000       1.347       2.805
trajectories_101              1.9979      0.605      3.304      0.001       0.813       3.183
trajectories_110              3.5740      0.712      5.019      0.000       2.178       4.970
trajectories_001:impmaize     0.5566      0.528      1.054      0.292      -0.479       1.592
trajectories_010:impmaize    -0.8352      0.556     -1.504      0.133      -1.924       0.254
trajectories_011:impmaize     0.9970      0.463      2.155      0.031       0.090       1.904
trajectories_100:impmaize    -1.3764      0.535     -2.570      0.010      -2.426      -0.327
trajectories_101:impmaize    -0.6308      0.714     -0.884      0.377      -2.030       0.768
trajectories_110:impmaize    -1.6275      0.680     -2.392      0.017      -2.961      -0.294
==============================================================================
Omnibus:                      613.335   Durbin-Watson:                   1.824
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              587.589
Skew:                           0.957   Prob(JB):                    2.55e-128
Kurtosis:                       2.226   Cond. No.                         12.6
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are robust to cluster correlation (cluster)
"""

In [ ]:
def weak_id_test(res, start=-100, stop=100, inc=0.1):
    trajectories = np.array(["010", "011", "100", "101", "110"])
    ranger = np.arange(start, stop, inc)
    mat = np.zeros((ranger.size, trajectories.size))
    for i, phi in enumerate(tqdm(ranger)):
        for j, traj in enumerate(trajectories):
            test = f"trajectories_{traj} - trajectories_001 = {phi}*(trajectories_{traj}:impmaize - trajectories_001:impmaize)"
            mat[i, j] = res.t_test(test).pvalue
            
    df= pd.DataFrame(columns = trajectories,
                 index = pd.Index(ranger),
                 data=mat)
            
    return df

def weak_id_joint_test(res, start=-100, stop=100, inc=0.1):
    
    trajectories = np.array(["010", "011", "100", "101", "110"])
    ranger = np.arange(start, stop, inc)
    mat = np.zeros(ranger.size)
    
    for i, phi in enumerate(tqdm(ranger)):
        joint_test_list = [f"(trajectories_{traj} - trajectories_001 = {phi}*(trajectories_{traj}:impmaize - trajectories_001:impmaize))" \
            for traj in trajectories]
    
        joint_test = ' , '.join(joint_test_list)
        mat[i] = res.f_test(joint_test).pvalue

    df= pd.DataFrame(columns = ['joint'],
                 index = pd.Index(ranger),
                 data=mat)
            
    return df

def phi_ci(weak_id_df):
    
    phi_p_min = weak_id_df[weak_id_df.apply(lambda x: x > 0.05)].min()
    phi_p_max = weak_id_df[weak_id_df.apply(lambda x: x > 0.05)].max()
    
    phi_df = pd.DataFrame(
        index = ['min', 'max'],
        columns = weak_id_df.columns
    )
    
    for col, mi, ma in zip(weak_id_df.columns, phi_p_min, phi_p_max):
        try:
            phi_df.loc['min', col] = weak_id_df.index[weak_id_df[col] == mi].values[0]
            phi_df.loc['max', col] = weak_id_df.index[weak_id_df[col] == ma].values[0]
        except IndexError:
            print(f"""Might be NaNs: 
                  phi_min = {phi_p_min.values[0]}
                  phi_max = {phi_p_max.values[0]}
                  """)
        
    return phi_df
        

In [ ]:
weak_id_joint = weak_id_test(reg_res_dict['YIELD_cropcutfresh'])

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
phi_ci(weak_id_joint)

,010,011,100,101,110
min,5.1,0.6,0.9,-0.8,-1.1
max,-0.2,-1.4,-0.1,0.2,-0.7


In [ ]:
weak_id_joint[weak_id_joint.apply(lambda x: x > 0.05)].

SyntaxError: invalid syntax (2050269669.py, line 1)